In [5]:
import os
import pickle
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from scipy.stats import pearsonr

In [6]:
def age_debiasing(predicted_ages, chronological_ages):
    
    predicted_ages = predicted_ages.to_numpy()
    chronological_ages = chronological_ages.to_numpy()
    
    ones = np.ones(len(chronological_ages))
    chronological_ages_squared = np.square(chronological_ages)
    chronological_ages_confounds = np.array([ones, chronological_ages])
    
    ones_predicted = np.ones(len(predicted_ages))
    predicted_ages_squared = np.square(predicted_ages)
    predicted_ages_confounds = np.array([ones_predicted, predicted_ages])
    
    predicted_ages = np.reshape(predicted_ages, (len(predicted_ages), 1) )
    beta = np.dot( np.linalg.pinv(chronological_ages_confounds.T), predicted_ages )

    corrected_predicted_ages = (predicted_ages.flatten() - beta[0])/beta[1]
    
    return corrected_predicted_ages

In [14]:
net58 = "58"
net74 = "74b"
net63 = "63"
net68 = "68"

net59 = "59"
net75 = "75b"
net64 = '64'
net69 = "69"

net60 = "60"
net76 = "76"
net65 = '65'
net70 = "70"

net61 = "61b"
net77 = "77b"
net66 = '66'
net71 = "71"

net62 = "62"
net78 = '78'
net67 = "67"
net72 = "72"

net73 = "73"







dfs = [
    net58,
    net74,
    net63,
    net68,
    
    net59,
    net75,
    net64,
    net69,
    
    net60,
    net76,
    net65,
    net70,
    
    net61,
    net77,
    net66,
    net71,
    
    net62,
    net78,
    net67,
    net72,
    net73,
]

names = [
    'p2_FA',           # 58
    'p2_FA_stepLR',           # 74
    'p5_FA',           # 63
    'p5_FA_noFinalLayer',           # 68
    
    'p2_TBSS_FA',      # 59
    'p2_TBSS_FA_stepLR',      # 75
    'p5_TBSS_FA',      # 64
    'p5_TBSS_FA_noFinalLayer',      # 69
    
    'p2_rsfmri_0',     # 60
    'p2_rsfmri_0_stepLR',     # 76
    'p5_rsfmri_0',     # 65
    'p5_rsfmri_0_noFinalLayer',     # 70
        
    'p2_SummedTracts',  # 61
    'p2_SummedTracts_stepLR',  # 77
    'p5_SummedTracts',  # 66
    'p5_SummedTracts_noFinalLayer',  # 71
    
    'p2_T2_Lession',  # 62
    'p2_T2_Lession_stepLR',  # 78
    'p5_T2_Lession',  # 67 
    'p5_T2_Lession_noFinalLayer',  # 72
    'p5_T1_stepLR', #73

]


dfs1, dfs2, dfs3 = [], [], []

dfs1 = dfs.copy()
# dfs2 = dfs.copy()
# dfs3 = dfs.copy()

for idx in range(len(dfs)):
    
#     if idx == 1:
#         dfs1[idx] = '../predictions/SM1-' + dfs1[idx] + '_predictions/output_statistics_test.csv'
#     else:
    dfs1[idx] = '../predictions/SM0-' + dfs1[idx] + '_predictions/output_statistics_test.csv'
#     dfs2[idx] = '../predictions/SM1-' + dfs2[idx] + '_predictions/output_statistics_test.csv'
#     dfs3[idx] = '../predictions/SM2-' + dfs3[idx] + '_predictions/output_statistics_test.csv'
    
for idx in range(len(dfs)):
    dfs1[idx] = pd.read_csv(dfs1[idx])
#     dfs2[idx] = pd.read_csv(dfs2[idx])
#     dfs3[idx] = pd.read_csv(dfs3[idx])

In [15]:
dfs = dfs1

In [16]:
dfs[0]

,Unnamed: 0,target_age,output_age,age_delta,loss
0,24642634,68.817620,68.704210,-0.113411,0.012862
1,25438579,54.541897,53.649475,-0.892422,0.796416
2,21026392,67.284820,63.460155,-3.824665,14.628063
3,22526044,67.492805,65.065620,-2.427185,5.891227
4,25008425,53.802400,59.677673,5.875275,34.518852
...,...,...,...,...,...
994,22396579,53.442810,51.903400,-1.539410,2.369782
995,21351411,52.529110,53.103943,0.574833,0.330433
996,21230487,53.252050,53.984417,0.732368,0.536364
997,22579199,64.787560,59.824326,-4.963234,24.633692


In [17]:
# for idx in range(len(dfs)):
#     dfs[idx] = pd.merge(dfs1[idx], dfs2[idx], on='Unnamed: 0', how='inner')
#     dfs[idx] = pd.merge(dfs[idx], dfs3[idx], on='Unnamed: 0', how='inner')
    
#     dfs[idx].rename({
#         'target_age_x': 'target_age_original',
#         'output_age_x': 'output_age_1',
#         'age_delta_x': 'age_delta_1',
#         'loss_x': 'loss_1',
#         'target_age_y': 'target_age_2',
#         'output_age_y': 'output_age_2',
#         'age_delta_y': 'age_delta_2',
#         'loss_y': 'loss_2',
#         'target_age': 'target_age_3',
#         'output_age': 'output_age_3',
#         'age_delta': 'age_delta_3',
#         'loss': 'loss_3',
#     }, axis=1, inplace=True)
    
#     dfs[idx].drop(['target_age_2', 'target_age_3'], axis=1, inplace=True)
    
#     dfs[idx].rename({
#         'target_age_original': 'target_age',
#     }, axis=1, inplace=True)
    
#     dfs[idx]['mean_output_age'] = dfs[idx][['output_age_1', 'output_age_2', 'output_age_3']].mean(axis=1)
#     dfs[idx]['mean_age_delta'] = dfs[idx]['mean_output_age'] - dfs[idx]['target_age']

In [18]:
# for idx in range(len(dfs)):    
#     dfs[idx]['output_age_decon'] = age_debiasing(dfs[idx].mean_output_age, dfs[idx].target_age) 
#     dfs[idx]['age_delta_decon'] = dfs[idx].output_age_decon - dfs[idx].target_age

for idx in range(len(dfs)):    
    dfs[idx]['output_age_decon'] = age_debiasing(dfs[idx].output_age, dfs[idx].target_age) 
    dfs[idx]['age_delta_decon'] = dfs[idx].output_age_decon - dfs[idx].target_age

In [19]:
# print("{:13} : {:14}  |  {:13}  ".format('Modality', 'MEAN (MAE)', "MEAN DECONF"))
# print('-----------------------------------------------------------------------------------------------')

# for idx in range(len(dfs)):
#     print("{:13} :  {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f} ".format(
#         names[idx],
#         np.abs(dfs[idx].mean_age_delta).mean(), np.abs(dfs[idx].mean_age_delta).std()/np.sqrt(len(dfs[idx])),
#         np.abs(dfs[idx].age_delta_decon).mean(), np.abs(dfs[idx].age_delta_decon).std()/np.sqrt(len(dfs[idx])),
#     )
#          )

print("{:30} : {:7}  |  {:13}  ".format('Modality', 'MEAN (MAE)', "MEAN DECONF"))
print('-----------------------------------------------------------------------------------------------')

for idx in range(len(dfs)):
    print("{:30} :  {:.3f}  |   {:.3f}  ".format(
        names[idx],
        np.abs(dfs[idx].age_delta).mean(), #np.abs(dfs[idx].age_delta).std()/np.sqrt(len(dfs[idx])),
        np.abs(dfs[idx].age_delta_decon).mean(), # np.abs(dfs[idx].age_delta_decon).std()/np.sqrt(len(dfs[idx])),
    )
         )
    
# T1 NonLinear:    2.858 | 2.994
# T2 NonLinear:    2.707 | 3.297


Modality                       : MEAN (MAE)  |  MEAN DECONF    
-----------------------------------------------------------------------------------------------
p2_FA                          :  3.369  |   4.030  
p2_FA_stepLR                   :  3.160  |   3.740  
p5_FA                          :  3.246  |   3.841  
p5_FA_noFinalLayer             :  3.270  |   3.945  
p2_TBSS_FA                     :  3.875  |   5.013  
p2_TBSS_FA_stepLR              :  4.062  |   5.450  
p5_TBSS_FA                     :  3.509  |   4.291  
p5_TBSS_FA_noFinalLayer        :  3.495  |   4.295  
p2_rsfmri_0                    :  5.381  |   10.996  
p2_rsfmri_0_stepLR             :  5.413  |   11.373  
p5_rsfmri_0                    :  5.502  |   12.204  
p5_rsfmri_0_noFinalLayer       :  5.395  |   10.896  
p2_SummedTracts                :  4.273  |   5.832  
p2_SummedTracts_stepLR         :  4.988  |   8.269  
p5_SummedTracts                :  3.874  |   4.998  
p5_SummedTracts_noFinalLayer   :  3.912  

In [9]:
print("{:20} : {:7}  |  {:7}  |  {:7}  |  {:7}".format('Modality', 'Run1', "Run2", "Run3", "MAE"))
print('-----------------------------------------------------------------------------------------------')

for idx in range(len(dfs)):
    print("{:20} : {:7.3f} |  {:7.3f}  |  {:7.3f} | {:7.3f}".format(
        names[idx],
        np.abs(dfs[idx].age_delta_1).mean(), 
        np.abs(dfs[idx].age_delta_2).mean(),
        np.abs(dfs[idx].age_delta_3).mean(), 
        np.abs(dfs[idx].mean_age_delta).mean(), 
    )
         )

Modality             : Run1     |  Run2     |  Run3     |  MAE    
-----------------------------------------------------------------------------------------------
3k_baseline          :   2.998 |    2.931  |    2.928 |   2.828
depth=(1,1,1,1)      :   2.890 |    2.939  |    2.983 |   2.818
heads=(2,4,8,16),b=4 :   2.951 |    3.096  |    3.072 |   2.901
feature_size=3       :   3.055 |    2.974  |    3.432 |   2.829
feature_size=6       :   3.097 |    3.145  |    3.034 |   2.887
feature_size=12      :   2.927 |    3.098  |    2.934 |   2.842
patch_size=3         :   3.034 |    3.074  |    3.314 |   2.938
patch_size=4         :   3.063 |    3.117  |    3.053 |   2.916
patch_size=5         :   3.012 |    3.006  |    3.105 |   2.847
warmup=0             :  63.414 |   63.559  |   63.287 |  63.420
warmup=8             :   2.918 |    2.982  |    3.050 |   2.830
warmup=12            :   2.973 |    3.085  |    3.018 |   2.866
weight_decay=1e-2    :   2.995 |    3.026  |    3.002 |   2.891
weigh

In [9]:
# Modality                : MEAN (MAE)      |  MEAN DECONF    
# -----------------------------------------------------------------------------------------------
# InputFusion             :  2.633 ± 0.063  |   2.906 ± 0.071 
# InputFusion_Filter      :  2.829 ± 0.069  |   3.267 ± 0.077 
# LayerFusion9600         :  2.794 ± 0.063  |   3.087 ± 0.073


mae_weighting = 37.137671232876755 # for female subjects
# # mae_weighting = 36.28150684931529 # for male subjects


print("{:20} : {:8}  |  {:7}   |  {:7}   |  {:7} ".format('Modality', 'MAE', "RMSE", 
                                                            "CORR (R)", "MAE weighted"))
print('-----------------------------------------------------------------------------------------------')

for idx in range(len(dfs)):
    print("{:20} :  {:7.3f}  |   {:7.3f}  |   {:7.3f}   |   {:7.3f} ".format(
        names[idx],
        np.abs(dfs[idx].mean_age_delta).mean(), 
        np.sqrt(np.square(np.subtract(dfs[idx].target_age, dfs[idx].mean_output_age)).mean()),
        pearsonr(dfs[idx].target_age, dfs[idx].mean_output_age)[0],
        np.abs(dfs[idx].mean_age_delta).mean() / mae_weighting,
    )
         )


Modality             : MAE       |  RMSE      |  CORR (R)   |  MAE weighted 
-----------------------------------------------------------------------------------------------
3k_baseline          :    2.828  |     3.565  |     0.878   |     0.076 
depth=(1,1,1,1)      :    2.818  |     3.552  |     0.880   |     0.076 
heads=(2,4,8,16),b=4 :    2.901  |     3.626  |     0.874   |     0.078 
feature_size=3       :    2.829  |     3.573  |     0.878   |     0.076 
feature_size=6       :    2.887  |     3.605  |     0.875   |     0.078 
feature_size=12      :    2.842  |     3.562  |     0.878   |     0.077 
patch_size=3         :    2.938  |     3.669  |     0.871   |     0.079 
patch_size=4         :    2.916  |     3.680  |     0.870   |     0.079 
patch_size=5         :    2.847  |     3.560  |     0.879   |     0.077 
warmup=0             :   63.420  |    63.855  |     0.003   |     1.708 
warmup=8             :    2.830  |     3.548  |     0.879   |     0.076 
warmup=12            :  

/Users/andreiroibu/opt/anaconda3/envs/SwinAgeMapper/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


In [10]:
maes=[]
prs = []
for idx in range(len(dfs)):
    mae = np.abs(dfs[idx].mean_age_delta).mean()
    maes.append(mae)
    pr = pearsonr(dfs[idx].target_age, dfs[idx].mean_output_age)[0]
    prs.append(pr)
    
print(pearsonr(maes, prs)[0])

ValueError: array must not contain infs or NaNs

In [30]:
print("{:20} : {:7}  |  {:7}  |  {:7} ".format('Modality', 'MAE (DECON)', "RMSE (DECON)", "PR (DECON)"))
print('-----------------------------------------------------------------------------------------------')

for idx in range(len(dfs)):
    print("{:20} :  {:10.3f}  |   {:10.3f}  |   {:7.3f} ".format(
        names[idx],
        np.abs(dfs[idx].age_delta_decon).mean(),
        np.sqrt(np.square(np.subtract(dfs[idx].target_age, dfs[idx].output_age_decon)).mean()), 
        pearsonr(dfs[idx].target_age, dfs[idx].output_age_decon)[0],
    )
         )

Modality             : MAE (DECON)  |  RMSE (DECON)  |  PR (DECON) 
-----------------------------------------------------------------------------------------------
3k_baseline          :       3.225  |        4.057  |     0.878 
depth=(1,1,1,1)      :       3.168  |        4.026  |     0.880 
heads=(2,4,8,16),b=4 :       3.294  |        4.148  |     0.874 
feature_size=3       :       3.214  |        4.058  |     0.878 
feature_size=6       :       3.237  |        4.108  |     0.875 
feature_size=12      :       3.230  |        4.055  |     0.878 
patch_size=3         :       3.361  |        4.208  |     0.871 
patch_size=4         :       3.341  |        4.226  |     0.870 
patch_size=5         :       3.213  |        4.041  |     0.879 
warmup=0             :    1758.911  |     2237.626  |     0.003 
warmup=8             :       3.195  |        4.036  |     0.879 
warmup=12            :       3.250  |        4.072  |     0.877 
weight_decay=1e-2    :       3.267  |        4.102  |   

# Saving Dataframe for Later Use

In [11]:
d = {'modality':names, 'dataframe':dfs}
df = pd.DataFrame(data=d)

In [12]:
df['dataframe'].iloc[0]

,Unnamed: 0,target_age,output_age_1,age_delta_1,loss_1,output_age_2,age_delta_2,loss_2,output_age_3,age_delta_3,loss_3,mean_output_age,mean_age_delta,output_age_decon,age_delta_decon
0,24642634,68.817620,71.761314,2.943695,8.665340,68.871870,0.054253,0.002943,70.051500,1.233879,1.522458,70.228228,1.410608,72.444804,3.627184
1,25438579,54.541897,53.974083,-0.567814,0.322413,55.331017,0.789120,0.622710,55.658980,1.117085,1.247878,54.988027,0.446130,52.637406,-1.904491
2,21026392,67.284820,63.286260,-3.998562,15.988497,61.453705,-5.831116,34.001910,62.329536,-4.955284,24.554840,62.356500,-4.928320,62.214070,-5.070750
3,22526044,67.492805,68.328830,0.836021,0.698932,66.923065,-0.569740,0.324604,67.739910,0.247101,0.061059,67.663935,0.171130,69.112041,1.619236
4,25008425,53.802400,65.822860,12.020462,144.491500,62.392452,8.590054,73.789020,63.143740,9.341343,87.260690,63.786351,9.983951,64.072419,10.270019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,22396579,53.442810,53.374626,-0.068184,0.004649,52.321102,-1.121708,1.258229,53.010162,-0.432648,0.187184,52.901963,-0.540847,49.926189,-3.516621
995,21351411,52.529110,53.400100,0.870991,0.758625,54.592310,2.063202,4.256802,51.478737,-1.050373,1.103284,53.157049,0.627939,50.257719,-2.271391
996,21230487,53.252050,54.929770,1.677723,2.814754,56.152126,2.900078,8.410451,54.943650,1.691601,2.861513,55.341849,2.089799,53.097261,-0.154789
997,22579199,64.787560,65.337270,0.549713,0.302185,63.529640,-1.257919,1.582361,65.086170,0.298607,0.089166,64.651027,-0.136533,65.196222,0.408662


In [13]:
df

,modality,dataframe
0,3k_baseline,Unnamed: 0 target_age output_age_1 age...
1,"depth=(1,1,1,1)",Unnamed: 0 target_age output_age_1 age...
2,"heads=(2,4,8,16),b=4",Unnamed: 0 target_age output_age_1 age...
3,feature_size=3,Unnamed: 0 target_age output_age_1 age...
4,feature_size=6,Unnamed: 0 target_age output_age_1 age...
5,feature_size=12,Unnamed: 0 target_age output_age_1 age...
6,patch_size=3,Unnamed: 0 target_age output_age_1 age...
7,patch_size=4,Unnamed: 0 target_age output_age_1 age...
8,patch_size=5,Unnamed: 0 target_age output_age_1 age...
9,warmup=0,Unnamed: 0 target_age output_age_1 age...


In [14]:
type(df.dataframe.iloc[0])

pandas.core.frame.DataFrame

In [15]:
df.iloc[15].dataframe

,Unnamed: 0,target_age,output_age_1,age_delta_1,loss_1,output_age_2,age_delta_2,loss_2,output_age_3,age_delta_3,loss_3,mean_output_age,mean_age_delta,output_age_decon,age_delta_decon
0,24642634,68.817620,70.556870,1.739250,3.024991,70.650300,1.832680,3.358715,69.216820,0.399200,0.159361,70.141330,1.323710,72.174465,3.356845
1,25438579,54.541897,58.433243,3.891346,15.142573,55.888706,1.346809,1.813896,56.256252,1.714355,2.939015,56.859400,2.317503,55.043345,0.501448
2,21026392,67.284820,62.173332,-5.111488,26.127314,61.572254,-5.712566,32.633415,61.430622,-5.854198,34.271640,61.725403,-5.559417,61.319547,-5.965273
3,22526044,67.492805,68.082670,0.589867,0.347943,68.300110,0.807304,0.651740,65.207900,-2.284905,5.220789,67.196893,-0.295912,68.376711,0.883906
4,25008425,53.802400,61.704834,7.902435,62.448483,61.479145,7.676746,58.932434,64.446150,10.643753,113.289480,62.543376,8.740976,62.374575,8.572175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,22396579,53.442810,53.910637,0.467827,0.218862,53.661915,0.219105,0.048007,55.336640,1.893829,3.586590,54.303064,0.860254,51.746166,-1.696644
995,21351411,52.529110,52.199177,-0.329933,0.108856,52.825070,0.295959,0.087592,53.102333,0.573223,0.328585,52.708860,0.179750,49.689951,-2.839159
996,21230487,53.252050,57.350590,4.098541,16.798040,56.105930,2.853882,8.144642,55.273760,2.021713,4.087325,56.243427,2.991377,54.248858,0.996808
997,22579199,64.787560,64.232666,-0.554894,0.307907,62.525090,-2.262470,5.118772,62.901756,-1.885803,3.556254,63.219837,-1.567723,63.247078,-1.540482


In [16]:
df.iloc[0].modality

'3k_baseline'

In [17]:
type(df.iloc[0].dataframe)

pandas.core.frame.DataFrame

In [18]:
df.to_pickle('female_test_prototype_repeat.pkl', protocol = 4)

In [50]:
# df2 = pd.read_pickle('female_test.pkl')

In [51]:
# df2

In [30]:
# df2.iloc[0].dataframe

In [31]:
# df2.iloc[0].modality

In [32]:
# type(df2.iloc[0].dataframe)

# BarPlots

In [33]:
# df_test = {}
# for idx in range(len(df)):
#     name = df.iloc[idx].modality
#     data = df.iloc[idx].dataframe.mean_output_age.to_list()
#     df_test[name] = data
    
# df_test = pd.DataFrame.from_dict(df_test)

# array = np.array(df_test)

# fig = plt.figure(figsize=(16, 7.2))
# plt.boxplot(array);
# plt.xticks(np.arange(1, array.shape[1]+1), df_test.columns.to_list(), rotation=90);

# plt.xlabel("Modalities")
# plt.ylabel("Predicted Age Distributions")
# plt.axvline(x=7.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=32.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=38.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=39.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=48.5, color='g', linestyle='dotted', alpha=0.5);

In [34]:
# df_test = {}
# for idx in range(len(df)):
#     name = df.iloc[idx].modality
#     data = df.iloc[idx].dataframe.mean_age_delta.to_list()
#     df_test[name] = data
    
# df_test = pd.DataFrame.from_dict(df_test)

# array = np.array(df_test)

# fig = plt.figure(figsize=(16, 7.2))
# plt.boxplot(array);
# plt.xticks(np.arange(1, array.shape[1]+1), df_test.columns.to_list(), rotation=90);

# plt.xlabel("Modalities")
# plt.ylabel("Predicted Age Delta Distributions")
# plt.axvline(x=7.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=32.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=38.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=39.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=48.5, color='g', linestyle='dotted', alpha=0.5);

In [35]:
# df_test = {}
# for idx in range(len(df)):
#     name = df.iloc[idx].modality
#     data = df.iloc[idx].dataframe.output_age_decon.to_list()
#     df_test[name] = data
    
# df_test = pd.DataFrame.from_dict(df_test)

# array = np.array(df_test)

# fig = plt.figure(figsize=(16, 7.2))
# plt.boxplot(array);
# plt.xticks(np.arange(1, array.shape[1]+1), df_test.columns.to_list(), rotation=90);

# plt.xlabel("Modalities")
# plt.ylabel("Predicted Debiased Age Distributions")
# plt.axvline(x=7.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=32.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=38.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=39.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=48.5, color='g', linestyle='dotted', alpha=0.5);

In [36]:
# df_test = {}
# for idx in range(len(df)):
#     name = df.iloc[idx].modality
#     data = df.iloc[idx].dataframe.age_delta_decon.to_list()
#     df_test[name] = data
    
# df_test = pd.DataFrame.from_dict(df_test)

# array = np.array(df_test)

# fig = plt.figure(figsize=(16, 7.2))
# plt.boxplot(array);
# plt.xticks(np.arange(1, array.shape[1]+1), df_test.columns.to_list(), rotation=90);

# plt.xlabel("Modalities")
# plt.ylabel("Predicted Debiased Age Deltas Distributions")
# plt.axvline(x=7.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=32.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=38.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=39.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=48.5, color='g', linestyle='dotted', alpha=0.5);

In [37]:
# df_test = {}
# for idx in range(len(df)):
#     name = df.iloc[idx].modality
#     data = df.iloc[idx].dataframe.age_delta_decon.to_list()
#     df_test[name] = data
    
# df_test = pd.DataFrame.from_dict(df_test)

# array = np.abs(np.array(df_test))

# fig = plt.figure(figsize=(16, 7.2))
# plt.boxplot(array);
# plt.xticks(np.arange(1, array.shape[1]+1), df_test.columns.to_list(), rotation=90);

# plt.xlabel("Modalities")
# plt.ylabel("Predicted Absolute Debiased Age Deltas Distributions")
# plt.axvline(x=7.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=32.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=38.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=39.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=48.5, color='g', linestyle='dotted', alpha=0.5);

In [38]:
# df_test = {}
# for idx in range(len(df)):
#     name = df.iloc[idx].modality
#     data = df.iloc[idx].dataframe.mean_age_delta.to_list()
#     df_test[name] = data
    
# df_test = pd.DataFrame.from_dict(df_test)

# array = np.array(df_test)

# fig = plt.figure(figsize=(16, 7.2))
# # plt.scatter(np.arange(1, len(array), 1), array);
# plt.scatter(np.arange(1, array.shape[1]+1), abs(array).mean(axis=0))
# plt.xticks(np.arange(1, array.shape[1]+1), df_test.columns.to_list(), rotation=90);

# plt.xlabel("Modalities")
# plt.ylabel("MAE")
# plt.axvline(x=7.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=32.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=38.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=39.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axvline(x=48.5, color='g', linestyle='dotted', alpha=0.5);
# plt.axhline(y=6.253, color='r', linestyle='dashed')

# Test what happens if debiasing before deconfounding

In [39]:
# dfs[0]

In [40]:
# for idx in range(len(dfs)):
#     dfs[idx]['output_age_decon_1'] = age_debiasing(dfs[idx].output_age_1, dfs[idx].target_age)
#     dfs[idx]['output_age_decon_2'] = age_debiasing(dfs[idx].output_age_2, dfs[idx].target_age) 
#     dfs[idx]['output_age_decon_3'] = age_debiasing(dfs[idx].output_age_3, dfs[idx].target_age) 
#     dfs[idx]['mean_output_age_decon'] = dfs[idx][['output_age_decon_1', 'output_age_decon_2', 'output_age_decon_3']].mean(axis=1)
#     dfs[idx]['mean_age_delta_decon'] = dfs[idx]['mean_output_age_decon'] - dfs[idx]['target_age']

In [41]:
# dfs[0]

In [42]:
# print("{:13} : {:14}  |  {:14}  |  {:13}  |   {:13}  |  {:}  |  {:} ".format('Modality', 'MAE (DECON)', 'MAE (DECON B)', "MSE (DECON)", "MSE (DECON B)",  "PR (DECON)", "PR (DECON B)"))
# print('-----------------------------------------------------------------------------------------------')

# for idx in range(len(dfs)):
#     print("{:13} :  {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f}   |   {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f}   |   {:.3f}   |   {:.3f} ".format(
#         names[idx],
        
#         np.abs(dfs[idx].age_delta_decon).mean(), np.abs(dfs[idx].age_delta_decon).std()/np.sqrt(len(dfs[idx])),
#         np.abs(dfs[idx].mean_age_delta_decon).mean(), np.abs(dfs[idx].mean_age_delta_decon).std()/np.sqrt(len(dfs[idx])),
        
#         np.square(np.subtract(dfs[idx].target_age, dfs[idx].output_age_decon)).mean(), np.square(np.subtract(dfs[idx].target_age, dfs[idx].output_age_decon)).std()/np.sqrt(len(dfs[idx])),
#         np.square(np.subtract(dfs[idx].target_age, dfs[idx].mean_output_age_decon)).mean(), np.square(np.subtract(dfs[idx].target_age, dfs[idx].mean_output_age_decon)).std()/np.sqrt(len(dfs[idx])),
        
#         pearsonr(dfs[idx].target_age, dfs[idx].output_age_decon)[0],
#         pearsonr(dfs[idx].target_age, dfs[idx].mean_output_age_decon)[0],
#     )
#          )